In [ ]:
%load_ext autoreload
%autoreload 2

## Imports

In [ ]:
from dmsbatch import create_batch_client, create_blob_client, query_yes_no

## First create a batch client from the config file

In [ ]:
client = create_batch_client('../tests/data/dmsbatch.config')   
blob_client = create_blob_client('../tests/data/dmsbatch.config')

## Application packages
To copy large files and programs it is best to zip (or targz) them and upload them as application packages

Application packages are setup separately in either azure management apis or from the web console or cli tool

These are referenced here by their name and version
e.g. DSM2, python and other programs

IWFM downloaded from https://data.cnra.ca.gov/dataset/iwfm-integrated-water-flow-model/resource/6be7a705-5577-4fa0-b02e-d28cabc75c9a and uploaded as an application package


In [ ]:
app_pkgs = [('unzip', '5.51-1', 'bin')]
# windows backslash needs to be doubled as per python string rules for escaping
iwfm_app_pkgs = [('IWFM','v2015.0.1129','IWFM 2015.0.1129\\iwfm-2015.0.1129_executables\\IWFM\\IWFM-2015.0.1129\\Budget'),
                ('IWFM','v2015.0.1129','IWFM 2015.0.1129\\iwfm-2015.0.1129_executables\\IWFM\\IWFM-2015.0.1129\\PreProcessor'),
                ('IWFM','v2015.0.1129','IWFM 2015.0.1129\\iwfm-2015.0.1129_executables\\IWFM\\IWFM-2015.0.1129\\Simulation'),
                ('IWFM','v2015.0.1129','IWFM 2015.0.1129\\iwfm-2015.0.1129_executables\\IWFM\\IWFM-2015.0.1129\\ZBudget')
                ]
#ignorning iwfm_app_packages as the version is available in side the zip file itself!

### Create or resize existing pool
If the pool doesn't exist it will create it
If the pool exists, it will resize to the second arg

Various VM sizes and performances are documented here :https://docs.microsoft.com/en-us/azure/virtual-machines/windows/compute-benchmark-scores

Pricing for these VMS can be found here : https://azure.microsoft.com/en-us/pricing/details/virtual-machines/windows/

Here are some results for 1973 - 1993 run of C2VSIM FG. v 1.0
 * 'standard_d4as_v4' took 2 hrs, 23 mins but had to go with higher CPU count as per cpu it is 8GB of temp storage
 * 'standard_f4s_v2' took 2 hrs, 40 mins with 8GB / vcpu of temp storage so higher CPU count was chosen
 * 'standard_d2d_v4', 4.5 hrs

In [ ]:
pool_name = 'C2VSimFGpool'

In [ ]:
client.create_or_resize_pool(pool_name,
                   1,
                   app_packages=[(app,version) for app,version,_ in app_pkgs], 
                   vm_size=  'standard_d2s_v5', 
                   tasks_per_vm=2,
                   os_image_data=('microsoftwindowsserver', 'windowsserver', '2019-datacenter-core')
                  )

### Create job on pool or fail if it exists
Jobs are containers of tasks (things that run on nodes (machines) in the pool). If this exists, the next line will fail

In [ ]:
job_name = 'c2vsimjob'

In [ ]:
try:
    client.create_job(job_name,pool_name)
except:
    print(f'Job {job_name} already exists. Continuing on ...')

### Upload input files
Zip the input files and add them to storage container (in this case the auto storage associated with the batch account)

In [ ]:
import os, datetime
userid = os.getlogin()
tsnow = str(datetime.datetime.now().timestamp()).split('.')[0]
task_name = f'c2vsim_{tsnow}_{userid}'
print(task_name)

C2VSimFG1.01 can be found https://data.cnra.ca.gov/dataset/c2vsimfg-version-1-01

In [ ]:
import os

In [ ]:
local_dir = f'../tests/data/c2vsimfg_v1_0_publicrelease/'
basefname = os.path.basename(os.path.normpath(local_dir))

zipfname=os.path.basename(os.path.normpath(local_dir))+'.zip'

In [ ]:
task_name,zipfname

In [ ]:
blob_client.zip_and_upload(job_name, f'{task_name}/{zipfname}', local_dir, max_concurrency=10)

In [ ]:
input_file = client.create_input_file_spec(job_name,blob_prefix=f'{task_name}/{zipfname}',file_path='.')

In [ ]:
output_dir_sas_url = blob_client.get_container_sas_url(job_name)  
output_dir_sas_url

In [ ]:
std_out_files = client.create_output_file_spec('../std*.txt', output_dir_sas_url, blob_path=f'{task_name}')
output_dir = client.create_output_file_spec('**/Results/*',output_dir_sas_url, blob_path=f'{task_name}')

### Create a task
This uses the application package as pre -set up. If not, create one https://docs.microsoft.com/en-us/azure/batch/batch-application-packages

In [ ]:
cmd_string = client.wrap_cmd_with_app_path([f'cd {task_name}',
                                            f'unzip {zipfname}',
                                            f'del {zipfname}',
                                            'cd Simulation',
                                           '..\\bin\\IWFM-2015.0.1045\\Simulation2015_x64.exe C2VSimFG.in'],
                                           app_pkgs)
print(cmd_string)
c2vsim_task = client.create_task(task_name, cmd_string, resource_files=[
                                input_file], output_files=[std_out_files, output_dir])

In [ ]:
task_name

### Next submit the task and wait 

In [ ]:
client.submit_tasks(job_name,[c2vsim_task])
#client.wait_for_tasks_to_complete(job_name,datetime.timedelta(minutes=30))

In [ ]:
client.wait_for_tasks_to_complete(job_name, datetime.timedelta(hours=3), polling_interval_secs=120)

In [ ]:
client.resize_pool(pool_name,0)

### Clean up Batch resources ( if the user so chooses)

In [ ]:
#if query_yes_no('Delete job?') == 'yes':
#    client.batch_client.job.delete(job_name)
#if query_yes_no('Delete pool?') == 'yes':
#    client.batch_client.pool.delete(pool_name)